# Load Mastodon tracking data on spot and branch level into Napari to use with Napari Clusters Plotter

## Prerequisites

Before running this notebook, the following steps need to be performed:
* Due to a bug in Napari clusters plotter handling tracking data, it may be required to install the latest version of Napari clusters plotter:
```
mamba activate devbio-napari
pip install git+https://github.com/BiAPoL/napari-clusters-plotter.git
```
* Compute features on Spot and BranchSpot Level in Mastodon
  * Open "Compute Features View"
  * Select all available Spot and Branch Spot features > Compute
    * Wait until computation is finished
* Compute relative movement for Spots and Branch Spots separately
  * Plugins > Compute feature > Movement of spots relative to nearest neighbors
* Export Spot and Branch Spot tables 
  * Open "Table View"
  * Select BranchSpot 
  * File > Export to CSV
  * Select Spot
  * File > Export to CSV 
* Export label image from Mastodon
  * Plugins > Exports > Export label image using ellipsoids
  * Select "Branch Spot ID" as label id
  * Select a frame reduction rate of 1
    * Higher frame reduction rates are possible and may speed up analyses in napari clusters plotter
    * If a frame reduction rate higher than 1 is used, a version of the intensity image would need to be created with the same frame reduction rate or no intensity image can be loaded into napari 

In [11]:
from pathlib import Path
import napari
import pandas as pd
from skimage.io import imread

## Read tables from Mastodon

#### Set path to tables
Enter the path to the tables here. Within that folder, there should be two tables:
* Spot.csv
* BranchSpot.csv

In [12]:
tables_folder_path = ''

#### Read tables from path
Rows with NaN values are removed.

In [13]:
spot_table_path = Path(tables_folder_path + '/Spot.csv')
spot_table = pd.read_csv(spot_table_path, skiprows=[1,2], encoding='utf-8', low_memory=False)
# remove rows with NaN values
spot_table = spot_table.dropna()

branch_spot_table_path = Path(tables_folder_path + '/BranchSpot.csv')
branch_spot_table = pd.read_csv(branch_spot_table_path, skiprows=[1,2], encoding='utf-8', low_memory=False)
# remove rows with NaN values
branch_spot_table = branch_spot_table.dropna()

#### Specify frame reduction factor
The frame reduction factor is the factor by which the frame rate has been reduced in Mastodon. This is necessary to account for the fact that the label image has been exported with a reduced frame rate, while the spot table has been exported with the original frame rate.
Specify here the same factor that has been used when exporting the label image from Mastodon

In [14]:
frame_reduction_factor = 1

#### Remove rows whose related frames are not in the label image

In [15]:
spot_table = spot_table[spot_table['Spot frame'] % frame_reduction_factor == 0]
spot_table['Spot frame'] = spot_table['Spot frame'] / frame_reduction_factor
spot_table['Spot frame'] = spot_table['Spot frame'].astype(int)

#### Optionally print head of spot table to check if everything is ok

In [16]:
spot_table.head(2)

,Label,ID,Branch spot ID,Spot N links,Spot N links.1,Spot N links.2,Spot center intensity,Spot ellipsoid aspect ratios,Spot ellipsoid aspect ratios.1,Spot ellipsoid aspect ratios.2,...,Detection.6,Division,Division.1,Division.2,Proliferator,Proliferator.1,Proliferator.2,Status,Tracking,Tracking.1
4,4,4,491,1,2,1,13743.196682,0.984351,0.943851,0.958856,...,0,0,0,0,1,0,0,0,1,0
16,16,16,2216,1,2,1,16971.599822,0.866112,0.506872,0.585227,...,0,0,0,0,0,0,1,0,1,0


#### Optionally print head of branch spot table to check if everything is ok

In [17]:
branch_spot_table.head(2)

,Label,ID,Branch Average Movement,Branch Movement Direction,Branch Movement Direction.1,Branch Movement Direction.2,Branch N Successors and Predecessors,Branch N Successors and Predecessors.1,Branch N leaves,Branch N spots,...,Detection.6,Division,Division.1,Division.2,Proliferator,Proliferator.1,Proliferator.2,Status,Tracking,Tracking.1
0,50644,0,0.845989,0.888329,0.458005,-0.033209,2,0,2,283,...,0,0,0,0,1,0,0,1,1,0
1,28749,1,0.785019,-0.840221,0.491131,-0.229825,2,0,2,176,...,0,0,0,0,1,0,0,1,1,0


## Change tables to match napari-clusters-plotter standards
Mastodon 'label' column needs to be removed from both tables. The spot table needs to be extended with a frame column and a label column.
The branch spot table needs to be extended with a label column.

In [18]:
# Remove Label column from Mastodon tables
spot_table = spot_table.drop(columns=['Label'])
branch_spot_table = branch_spot_table.drop(columns=['Label'])

# Add frame and label column to spot table
spot_table['frame'] = spot_table['Spot frame'].astype(int)
spot_table['label'] = spot_table['Branch spot ID'].astype(int) + 1 # Turning branch spot ids into labels, NB: + 1 needs to be added, since the ids are counted one based in the respective Mastodon export plugin

# Add a column 'Branch spot ID' to allow table merging
branch_spot_table['label'] = branch_spot_table['ID'].astype(int) + 1 # Turning branch spot ids into labels, NB: + 1 needs to be added, since the ids are counted one based in the respective Mastodon

### Currently available Branch spot features:
* label
* Branch Average Movement
* Branch Movement Direction (X)
* Branch Movement Direction.1 (Y)
* Branch Movement Direction.2 (Z)
* Branch N Successors and Predecessors (successors)
* Branch N Successors and Predecessors.1 predecessors (predecessors)
* Branch Sinuosity
* Branch duration and displacement (displacement)
* Branch duration and displacement.1 (duration)
* Branch Relative Movement (x-component)
* Branch Relative Movement.1 (y-component)
* Branch Relative Movement.2 (z-component)
* Branch Relative Movement.3 (average speed relative to 5 nearest neighbors)
* Cell Division Frequency
### Currently available Branch spot features:
* label 
* Spot center intensity
* Spot ellipsoid aspect ratios (a_b)
* Spot ellipsoid aspect ratios.1 (a_c)
* Spot ellipsoid aspect ratios.2 (b_c)
* Spot ellipsoid properties (a)
* Spot ellipsoid properties.1 (b)
* Spot ellipsoid properties.2 (c)
* Spot ellipsoid properties.3 (v)
* Spot frame (frame)
* Spot intensity (mean)
* Spot intensity.1 (std)
* Spot intensity.2 (min)
* Spot intensity.3 (max)
* Spot intensity.4 (median)
* Spot intensity.5 (sum)
* Spot position (x)
* Spot position.1 (y)
* Spot position.2 (z)
* Spot relative movement (x)
* Spot relative movement.1 (y)
* Spot relative movement.2 (z)
* Spot relative movement.3 (distance)

### Merge spot table and branch spot table

In [19]:
measurements_temp = pd.merge(left=spot_table, right=branch_spot_table, how='outer', on='label', suffixes=('_spot', '_branch'))
measurements_temp.head(2)

,ID_spot,Branch spot ID,Spot N links,Spot N links.1,Spot N links.2,Spot center intensity,Spot ellipsoid aspect ratios,Spot ellipsoid aspect ratios.1,Spot ellipsoid aspect ratios.2,Spot ellipsoid properties,...,Detection.6_branch,Division_branch,Division.1_branch,Division.2_branch,Proliferator_branch,Proliferator.1_branch,Proliferator.2_branch,Status_branch,Tracking_branch,Tracking.1_branch
0,4,491,1,2,1,13743.196682,0.984351,0.943851,0.958856,3.218713,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,26323,491,1,2,1,20353.173259,0.710119,0.640163,0.901486,2.344018,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Optional create a cell fate column 
This is only useful, if the cell fate has been annotated in Mastodon.

In [10]:
# Define a function to determine the combined value
def cell_fate_values_to_label(row):
    if row['cell_fate_spot']:
        return 1
    elif row['cell_fate.1_spot']:
        return 2
    elif row['cell_fate.2_spot']:
        return 3
    elif row['cell_fate.3_spot']:
        return 4
    elif row['cell_fate.4_spot']:
        return 5
    elif row['cell_fate.5_spot']:
        return 6
    elif row['cell_fate.6_spot']:
        return 7
    elif row['cell_fate.7_spot']:
        return 8
    elif row['cell_fate.8_spot']:
        return 9
    elif row['cell_fate.9_spot']:
        return 10
    elif row['cell_fate.10_spot']:
        return 11
    elif row['cell_fate.11_spot']:
        return 12
    elif row['cell_fate.12_spot']:
        return 13
    else:
        return 0 

# create a new column with 13 different cell fates
measurements_temp['cell_fate'] = measurements_temp.apply(cell_fate_values_to_label, axis=1)

### Remove unnecessary columns to save some RAM

In [20]:
columns_to_keep = ['label', 'Branch Average Movement', 'Branch Movement Direction', 'Branch Movement Direction.1', 'Branch Movement Direction.2', 'Branch N Successors and Predecessors', 'Branch N Successors and Predecessors.1', 'Branch Sinuosity', 'Branch duration and displacement', 'Branch duration and displacement.1', 'Branch relative movement', 'Branch relative movement.1', 'Branch relative movement.2', 'Branch relative movement.3', 'Cell division frequency', 'Spot center intensity', 'Spot ellipsoid aspect ratios', 'Spot ellipsoid aspect ratios.1', 'Spot ellipsoid aspect ratios.2', 'Spot ellipsoid properties', 'Spot ellipsoid properties.1', 'Spot ellipsoid properties.2', 'Spot ellipsoid properties.3', 'Spot intensity', 'Spot intensity.1', 'Spot intensity.2', 'Spot intensity.3', 'Spot intensity.4', 'Spot intensity.5', 'Spot position', 'Spot position.1', 'Spot position.2', 'Spot relative movement', 'Spot relative movement.1', 'Spot relative movement.2', 'Spot relative movement.3' ]
if 'cell_fate' in measurements_temp.columns:
    columns_to_keep.append('cell_fate')
measurements = measurements_temp[columns_to_keep]
measurements.head(2)


,label,frame,Branch Average Movement,Branch Movement Direction,Branch Movement Direction.1,Branch Movement Direction.2,Branch N Successors and Predecessors,Branch N Successors and Predecessors.1,Branch Sinuosity,Branch duration and displacement,...,Spot intensity.3,Spot intensity.4,Spot intensity.5,Spot position,Spot position.1,Spot position.2,Spot relative movement,Spot relative movement.1,Spot relative movement.2,Spot relative movement.3
0,492,225,0.911063,0.725417,-0.233526,0.647484,0.0,2.0,25.682919,12.775965,...,38852.0,13178.0,13248596.0,76.696254,60.373360,8.583054,-0.251165,-0.436157,0.073730,0.508677
1,492,179,0.911063,0.725417,-0.233526,0.647484,0.0,2.0,25.682919,12.775965,...,61690.0,16512.0,13525549.0,78.669836,72.730003,4.960000,-2.231562,0.423914,1.047295,2.501279


### Rename columns to have more meaningful names

In [21]:
new_columns = {'Branch duration and displacement.1': 'Branch duration', 'Branch duration and displacement': 'Branch displacement', 'Branch Movement Direction' : 'Branch Movement Direction (X)', 'Branch Movement Direction.1' : 'Branch Movement Direction (Y)', 'Branch Movement Direction.2' : 'Branch Movement Direction (Z)', 'Branch relative movement' : 'Branch Relative Movement (x-component)', 'Branch relative movement.1' : 'Branch Relative Movement (y-component)', 'Branch relative movement.2' : 'Branch Relative Movement (z-component)', 'Branch relative movement.3' : 'Branch Relative Movement', 'Branch N Successors and Predecessors' : 'Branch N Successors', 'Branch N Successors and Predecessors.1' : 'Branch N Predecessors', 'Spot ellipsoid aspect ratios': 'Spot ellipsoid aspect ratio a_b', 'Spot ellipsoid aspect ratios.1': 'Spot ellipsoid aspect ratio a_c', 'Spot ellipsoid aspect ratios.2': 'Spot ellipsoid aspect ratio b_c', 'Spot ellipsoid properties': 'Spot ellipsoid axis a', 'Spot ellipsoid properties.1': 'Spot ellipsoid axis b', 'Spot ellipsoid properties.2': 'Spot ellipsoid axis c', 'Spot ellipsoid properties.3': 'Spot ellipsoid axis v', 'Spot intensity' : 'Spot intensity (mean)', 'Spot intensity.1' : 'Spot intensity (std)', 'Spot intensity.2' : 'Spot intensity (min)', 'Spot intensity.3' : 'Spot intensity (max)', 'Spot intensity.4' : 'Spot intensity (median)', 'Spot intensity.5' : 'Spot intensity (sum)', 'Spot position' : 'Spot position x', 'Spot position.1' : 'Spot position y', 'Spot position.2' : 'Spot position z', 'Spot relative movement' : 'Spot relative movement x', 'Spot relative movement.1' : 'Spot relative movement y', 'Spot relative movement.2' : 'Spot relative movement z', 'Spot relative movement.3' : 'Spot relative movement distance' }

if 'cell_fate' in measurements.columns:
    new_columns['cell_fate'] = 'Cell fate_CLUSTER_ID'

# Rename the columns using the dictionary
measurements.rename(columns=new_columns, inplace=True)
measurements.head(2)

C:\Users\stha735e\AppData\Local\Temp\ipykernel_25760\524075988.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurements.rename(columns=new_columns, inplace=True)


,label,frame,Branch Average Movement,Branch Movement Direction (X),Branch Movement Direction (Y),Branch Movement Direction (Z),Branch N Successors,Branch N Predecessors,Branch Sinuosity,Branch displacement,...,Spot intensity (max),Spot intensity (median),Spot intensity (sum),Spot position x,Spot position y,Spot position z,Spot relative movement x,Spot relative movement y,Spot relative movement z,Spot relative movement distance
0,492,225,0.911063,0.725417,-0.233526,0.647484,0.0,2.0,25.682919,12.775965,...,38852.0,13178.0,13248596.0,76.696254,60.373360,8.583054,-0.251165,-0.436157,0.073730,0.508677
1,492,179,0.911063,0.725417,-0.233526,0.647484,0.0,2.0,25.682919,12.775965,...,61690.0,16512.0,13525549.0,78.669836,72.730003,4.960000,-2.231562,0.423914,1.047295,2.501279


### Optionally export measurements to CSV file
This can be skipped if the measurements are not needed outside napari.

In [13]:
measurements.to_csv(tables_folder_path + '/' + 'measurements_spot_branch_spot_' + str(frame_reduction_factor) + '.csv', sep=',', quotechar='"', index=False)

## View in napari
* Installation instructions for napari can be found [here](https://biapol.github.io/blog/mara_lampert/getting_started_with_mambaforge_and_python/readme.html).

### Read label image
The label image is expected to be exported from Mastodon with the following settings:
* Label Id: *Branch spot ID*
* Frame rate reduction: expected to be the same as the frame reduction factor specified above

#### Set path to label image
Enter the path to the label image exported from Mastodon here.

In [22]:
label_image_path = ''

#### Read label image from path

In [23]:
#### Read label image from path
label_image_path = Path(label_image_path)
label_image = imread(label_image_path)

#### Optionally print shape of label image to check if everything is ok, order: t, z, y, x

In [24]:
print(label_image.shape)

(504, 12, 500, 1024)


### Optionally read intensity image
This will only work, if the intensity image has the same frame reduction rate as the label image.

#### Optionally set path to intensity image
Enter the path to the intensity image here.

In [17]:
intensity_image_path = ''

#### Optionally read intensity image from path

In [18]:
intensity_image_path = Path(intensity_image_path)
intensity_image = imread(intensity_image_path)

#### Optionally print shape of intensity image to check if everything is ok. Order: t, z, y, x

In [19]:
print(intensity_image.shape)

(504, 12, 500, 1024)


### Open napari viewer

In [25]:
viewer = napari.Viewer()

### Set scale of intensity image
Due to bugs both in Mastodon export and in Napari import scale needs to be set manually.
Expected order: t, z, y, x

### Add label image

In [27]:
labels_layer = viewer.add_labels(label_image, features=measurements)

### Set scale of label image
Due to bugs both in Mastodon export and in Napari import scale needs to be set manually.
Expected order: t, z, y, x

In [28]:
# labels_layer.scale = (1, 2.48, 0.31196313094933187, 0.31196313094933187)
# labels_layer.scale = (1, 2.03, 0.41, 0.41)
# set scale in napari terminal
# viewer.layers[0].scale = (1, 2.48, 0.31196313094933187, 0.31196313094933187)

### Optionally add intensity image

In [ ]:
intensity_layer = viewer.add_image(intensity_image)

### Optionally set scale of intensity image
Due to bugs both in Mastodon export and in Napari import scale needs to be set manually.
Expected order: t, z, y, x
Should be the same as for the label image.

In [ ]:
# intensity_layer.scale = (1, 2.48, 0.31196313094933187, 0.31196313094933187)
# intensity_layer.scale = (1, 2.03, 0.41, 0.41)

### Turn on 3D view

In [29]:
viewer.dims.ndisplay = 3

### Load napari-clusters-plotter plugin

In [30]:
viewer.window.add_plugin_dock_widget(plugin_name='napari-clusters-plotter', widget_name='Plotter Widget')


(<napari._qt.widgets.qt_viewer_dock_widget.QtViewerDockWidget at 0x24cd4362550>,
 <napari_clusters_plotter._plotter.PlotterWidget at 0x249b3f0b1f0>)

## You are ready to use the napari-clusters-plotter with the Mastodon data plugin now.
Consult the documentation of the napari-clusters-plotter for further instructions, if needed.
* https://www.youtube.com/watch?v=qZ8KDrgL1Ro
* https://github.com/BiAPoL/napari-clusters-plotter